# Recommender Systems and Choice: A Reproducibility Study

*by Adeline Liem*


## Data Wrangling

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import SnowballStemmer
from scipy import stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
import os

In [2]:
# Read datasets

kilands_df = pd.read_csv('kilands_20201103.csv')
cyberfoto_df = pd.read_csv('cyberfoto_20210121.csv')

In [3]:
kilands_df.shape

(14832, 26)

In [4]:
cyberfoto_df.shape

(37005, 24)

In [5]:
kilands_df.describe()

,number of add-to-carts,number of clicks,first click rank,first purchase rank,order value,number of purchases,number of searches in current session,search session length (seconds),number of products displayed to user,number of sessions,whole session length,time to add-to-cart (seconds),time to first click (seconds),time to purchase
count,14832.000000,14832.000000,7422.000000,249.000000,249.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,14832.000000,657.000000,7422.000000,249.000000
mean,0.050499,0.982470,15.801536,12.887550,1638.630602,0.018406,2.158306,151.272923,42.654396,4.500202,514.805218,266.042618,36.330369,471.072289
std,0.258518,1.763807,40.195333,34.877369,2144.580567,0.146424,1.798311,1828.612696,66.212976,13.663486,6762.696042,289.368074,79.857679,360.449835
min,0.000000,0.000000,0.000000,0.000000,39.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,3.000000,1.000000,40.000000
25%,0.000000,0.000000,1.000000,0.000000,409.500000,0.000000,1.000000,0.000000,5.000000,1.000000,18.000000,71.000000,8.000000,195.000000
50%,0.000000,1.000000,4.000000,1.000000,985.000000,0.000000,2.000000,20.000000,24.000000,1.000000,96.000000,157.000000,17.000000,369.000000
75%,0.000000,1.000000,15.000000,7.000000,2090.000000,0.000000,3.000000,92.000000,48.000000,2.000000,344.000000,338.000000,36.000000,655.000000
max,6.000000,50.000000,1634.000000,321.000000,16490.000000,3.000000,14.000000,155064.000000,1416.000000,97.000000,336633.000000,1747.000000,1507.000000,1777.000000


In [11]:
# Removing outliers

# Defining bounds for exclusion: search session length

upper_bound_length_kilands = kilands_df['search session length (seconds)'].mean() + \
                             2.5 * kilands_df['search session length (seconds)'].std()
print(upper_bound_length_kilands)

# Count the number of observations with search session length exceeding the upper bound
num_outliers_search = len(kilands_df[kilands_df['search session length (seconds)'] > upper_bound_length_kilands])
print(num_outliers_search)

# Creating new dataset with search session outliers removed
kilands_no_out = kilands_df[kilands_df['search session length (seconds)'] <= upper_bound_length_kilands]

4722.8046632183505
27


In [14]:
# Define bounds for exclusion: clicks
upper_bound_clicks_kilands = kilands_df['number of clicks'].mean() + \
                             2.5 * kilands_df['number of clicks'].std()
print(upper_bound_clicks_kilands)

# Count the number of observations with clicks exceeding the upper bound
num_outliers_clicks = len(kilands_no_out[kilands_no_out['number of clicks'] > upper_bound_clicks_kilands])
print(num_outliers_clicks)

# Updating new dataset with click outliers removed
kilands_no_out = kilands_df[kilands_df['number of clicks'] <= upper_bound_clicks_kilands]

5.3919867279927525
374


In [15]:
kilands_no_out

,search id,user,session,segment,query,slot,platform,search start,search end time,number of add-to-carts,...,search session length (seconds),number of products displayed to user,number of sessions,whole session length,time to add-to-cart (seconds),time to first click (seconds),time to purchase,toggled filters,click positions,purchase positions
0,u:004f5XJdPvnue61i/s:IFEonOw2XywDPmFc/q:q=sisa...,004f5XJdPvnue61i,IFEonOw2XywDPmFc,202004_kilands_shuffle7,sisal,NaN,mobile,2020-05-11T13:48:33.391,2020-05-11T13:48:33.391,0,...,32,24,1,32,NaN,32.0,NaN,NaN,23,NaN
1,u:00U825g1ZwBVX8UW/s:tQCDykE6URsuP1t7/q:q=bilm...,00U825g1ZwBVX8UW,tQCDykE6URsuP1t7,202004_kilands_shuffle7,Bilmatta,NaN,mobile,2020-04-26T06:38:01.135,2020-04-26T06:39:13.998,0,...,73,8,1,73,NaN,13.0,NaN,NaN,0;0,NaN
2,u:01U04AG9RtpnKveG/s:r0I5bjEYaljUF8Ff/q:q=tras...,01U04AG9RtpnKveG,r0I5bjEYaljUF8Ff,202004_kilands_shuffle7,trasmatt,NaN,desktop,2020-05-03T21:18:20.127,2020-05-03T21:18:20.127,0,...,17,1,1,17,NaN,17.0,NaN,NaN,0,NaN
3,u:01WAyTwhvMkQVpSv/s:HIrTC5hOFuzum34m/q:q=matt...,01WAyTwhvMkQVpSv,HIrTC5hOFuzum34m,202004_gandalf_rel,runda mattor,NaN,mobile,2020-04-18T15:28:23.350,2020-04-18T15:28:23.350,0,...,12,24,1,12,NaN,12.0,NaN,NaN,8,NaN
4,u:022YZKPnqb7KERVw/s:53Z43z9957vgQKIY/q:q=plas...,022YZKPnqb7KERVw,53Z43z9957vgQKIY,202004_gandalf_rel,plastmattor,NaN,mobile,2020-04-29T23:18:04.324,2020-04-29T23:18:33.885,0,...,39,72,1,39,NaN,39.0,NaN,NaN,53,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14827,u:zxw5bXtZLcAzBAPs/s:SVKHkRkMD5Z5dKMh/q:q=rund...,zxw5bXtZLcAzBAPs,SVKHkRkMD5Z5dKMh,202004_gandalf_rel,Rund,NaN,tablet,2020-05-08T22:48:41.826,2020-05-08T22:49:42.753,0,...,61,120,1,71,NaN,NaN,NaN,NaN,NaN,NaN
14828,u:zxw5bXtZLcAzBAPs/s:SVKHkRkMD5Z5dKMh/q:q=matt...,zxw5bXtZLcAzBAPs,SVKHkRkMD5Z5dKMh,202004_gandalf_rel,rund matta,NaN,tablet,2020-05-08T22:49:49.275,2020-05-08T22:49:52.962,0,...,4,48,1,71,NaN,NaN,NaN,NaN,NaN,NaN
14829,u:zyfLPL38d2wolWFK/s:iKyf7aO0Ta2asuqP/q:q=274 ...,zyfLPL38d2wolWFK,iKyf7aO0Ta2asuqP,202004_gandalf_rel,bristol 274,NaN,desktop,2020-04-19T19:57:44.909,2020-04-19T19:57:44.909,0,...,33,1,1,33,NaN,33.0,NaN,NaN,0,NaN
14830,u:zz4XT5jMM6FMGos0/s:ClDePgytzZ4L4MGK/q:q=rio/...,zz4XT5jMM6FMGos0,ClDePgytzZ4L4MGK,202004_kilands_shuffle7,rio,NaN,tablet,2020-04-29T01:51:45.688,2020-04-29T01:52:03.500,1,...,18,4,1,33,5.0,5.0,NaN,NaN,2,NaN


## Project Progress Check 2
